In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-11-01'
final_date = '2022-11-30'
query_error_year = 2022
query_error_month = 11

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\11. NOVIEMBRE\DATA\ERROR NOVIEMBRE 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(111959, 17)

In [5]:
query_error['mes_cierre'].unique()

array([nan, 'Actual'], dtype=object)

In [6]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [7]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [8]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [9]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_11156\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
101103,101103,ee8c6855-c796-4db9-81f7-8534f353aa5b,Takos De kanasta Rios,LKHX3J5MR,2022-11-01,90.08,ERROR,R8,2022-11-01,11,2022,NEW,ERROR_USUAL,ee8c6855-c796-4db9-81f7-8534f353aa5b,2022-11-01,1,Actual
99677,99677,020a0ad3-4e44-426a-ad94-3b9800d457fd,Miscelanea Karla,L2QTBTH2D,2022-10-28,181.11,ERROR,Error request by Backoffice,2022-11-01,11,2022,ERROR,ERROR_USUAL,020a0ad3-4e44-426a-ad94-3b9800d457fd,2022-11-01,1,Actual
99659,99659,d48beed5-51e0-43d7-84c8-57754b61f4a2,SUPERTICK,KQ32L6S5Z,2022-10-28,127.49,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,d48beed5-51e0-43d7-84c8-57754b61f4a2,2022-11-01,1,Actual
99657,99657,5665e03f-407f-49a8-bf85-ff4e28bf862d,City Cell San Nicolas,24T7JZ4J9,2022-10-28,400.62,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,5665e03f-407f-49a8-bf85-ff4e28bf862d,2022-11-01,1,Actual
99656,99656,36c02f4b-22c4-4357-8ae8-bd9071793a92,Wet Cleaning,2ZCQC564Q,2022-10-28,480.16,ERROR,Error request by Backoffice,2022-11-01,11,2022,NEW,ERROR_USUAL,36c02f4b-22c4-4357-8ae8-bd9071793a92,2022-11-01,1,Actual


In [10]:
query_error_current_month.shape

(11341, 17)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# AGRUPADO POR ESQUEMA

In [14]:
group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only      amount                       \
esquema_query                      ERROR_USUAL            SIN_CLABE   
status_cubeta                            ERROR        NEW     ERROR   
0                       2022-11-01   21,600.08  94,616.59    137.99   
1                       2022-11-02        0.00  13,196.50    163.85   
2                       2022-11-03  186,171.31 135,049.35      0.00   
3                       2022-11-04    6,955.25 111,723.82      0.00   
4                       2022-11-05        0.00   2,923.66      0.00   

                                     
esquema_query                   All  
status_cubeta        NEW             
0             122,069.19 238,423.85  
1              49,185.51  62,545.86  
2              62,733.95 383,954.61  
3              95,311.21 213,990.28  
4              68,961.61  71,885.27

In [15]:
group_esquema.shape

(31, 6)

In [16]:
writer = pd.ExcelWriter('FASE VI. DATA POLIZA ERROR-SIN CLABE NOVIEMBRE 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
query_error_current_month.to_excel(writer, sheet_name='DETALLE',index=False)
writer.close()